In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
test=pd.read_csv('C:/Users/yifeichongtian/Desktop/UCL course material/web economics/report/dataset/test.csv')

In [3]:
validation=pd.read_csv('C:/Users/yifeichongtian/Desktop/UCL course material/web economics/report/dataset/validation.csv')

In [4]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299749 entries, 0 to 299748
Data columns (total 26 columns):
click             299749 non-null int64
weekday           299749 non-null int64
hour              299749 non-null int64
bidid             299749 non-null object
logtype           299749 non-null int64
userid            299749 non-null object
useragent         299749 non-null object
IP                299749 non-null object
region            299749 non-null int64
city              299749 non-null int64
adexchange        299749 non-null object
domain            299749 non-null object
url               299749 non-null object
urlid             299749 non-null object
slotid            299749 non-null object
slotwidth         299749 non-null int64
slotheight        299749 non-null int64
slotvisibility    299749 non-null object
slotformat        299749 non-null object
slotprice         299749 non-null int64
creative          299749 non-null object
bidprice          299749 non-null int

In [5]:
new_validation=validation['bidprice']
new_validation=pd.DataFrame(validation['bidprice'])
new_validation.columns=['bidprice']

In [6]:
new_validation.bidprice.value_counts()

300    115221
294     39256
238     38786
227     31467
241     25098
277     22199
254     14636
249     13086
Name: bidprice, dtype: int64

In [8]:
validation.bidprice.value_counts()

300    115221
294     39256
238     38786
227     31467
241     25098
277     22199
254     14636
249     13086
Name: bidprice, dtype: int64

In [9]:
validation.adexchange.value_counts()

3       99896
2       92986
1       91118
4        9715
null     6034
Name: adexchange, dtype: int64

In [10]:
validation.loc[validation.adexchange=='3','adexchange':]=3
validation.loc[validation.adexchange=='2','adexchange':]=2
validation.loc[validation.adexchange=='1','adexchange':]=1
validation.loc[validation.adexchange=='4','adexchange':]=4
validation.loc[validation.adexchange=='null','adexchange':]=0

In [11]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [12]:
validation['adexchange'].value_counts()

3    99896
2    92986
1    91118
4     9715
0     6034
Name: adexchange, dtype: int64

In [13]:
validation['userid']=label_encoder.fit_transform(validation['userid'])
validation['useragent']=label_encoder.fit_transform(validation['useragent'])
validation['usertag']=label_encoder.fit_transform(validation['usertag'])
validation['adexchange']=label_encoder.fit_transform(validation['adexchange'])
validation['IP']=label_encoder.fit_transform(validation['IP'])

'hour','weekday','hour','useragent','IP','adexchange','slotwidth','slotheight',

In [14]:
validation['intercept']=1
feature=['click','useragent','usertag','userid','weekday','hour','intercept','adexchange','weekday','hour','city','region']
validation_sample=validation[feature]

In [15]:
validation_sample.corr(method='spearman')

,click,useragent,usertag,userid,weekday,hour,intercept,adexchange,weekday,hour,city,region
click,1.000000,-0.009471,-0.005752,-0.001800,0.001420,0.001790,NaN,-0.005752,0.001420,0.001790,-0.000419,-0.000244
useragent,-0.009471,1.000000,0.043784,0.100422,-0.008815,-0.033304,NaN,0.043784,-0.008815,-0.033304,0.019590,0.017450
usertag,-0.005752,0.043784,1.000000,-0.014228,-0.092711,0.041467,NaN,1.000000,-0.092711,0.041467,-0.039271,-0.039714
userid,-0.001800,0.100422,-0.014228,1.000000,0.000220,-0.020138,NaN,-0.014228,0.000220,-0.020138,-0.057944,-0.057178
weekday,0.001420,-0.008815,-0.092711,0.000220,1.000000,-0.108740,NaN,-0.092711,1.000000,-0.108740,0.011499,0.011219
hour,0.001790,-0.033304,0.041467,-0.020138,-0.108740,1.000000,NaN,0.041467,-0.108740,1.000000,-0.037935,-0.038821
intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adexchange,-0.005752,0.043784,1.000000,-0.014228,-0.092711,0.041467,NaN,1.000000,-0.092711,0.041467,-0.039271,-0.039714
weekday,0.001420,-0.008815,-0.092711,0.000220,1.000000,-0.108740,NaN,-0.092711,1.000000,-0.108740,0.011499,0.011219
hour,0.001790,-0.033304,0.041467,-0.020138,-0.108740,1.000000,NaN,0.041467,-0.108740,1.000000,-0.037935,-0.038821


In [16]:
x_validation=validation_sample.drop(['click'],axis=1)

In [17]:
validation.click.value_counts()

0    299523
1       226
Name: click, dtype: int64

# Decision tree

In [18]:
decision_tree=DecisionTreeClassifier()
decision_tree.fit(x_validation, validation.click)
decision_tree.score(x_validation, validation.click)
prediction=decision_tree.predict(x_validation)
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

0    299523
1       226
Name: click, dtype: int64

In [19]:
test.loc[test.adexchange=='3','adexchange':]=3
test.loc[test.adexchange=='2','adexchange':]=2
test.loc[test.adexchange=='1','adexchange':]=1
test.loc[test.adexchange=='4','adexchange':]=4
test.loc[test.adexchange=='null','adexchange':]=0

In [20]:
test['userid']=label_encoder.fit_transform(test['userid'])
test['useragent']=label_encoder.fit_transform(test['useragent'])
test['usertag']=label_encoder.fit_transform(test['usertag'])
test['adexchange']=label_encoder.fit_transform(test['adexchange'])

In [21]:
test['intercept']=1
feature=['useragent','usertag','userid','weekday','hour','intercept','adexchange','weekday','hour','city','region']
test_sample=test[feature]

In [22]:
prediction_test=decision_tree.predict(test_sample)
prediction_test=pd.DataFrame(prediction_test)
prediction_test.columns=['click']
prediction_test.click.value_counts()

0    299422
1       327
Name: click, dtype: int64

In [23]:
Difference=prediction_test.click-prediction.click
Difference=pd.DataFrame(Difference)
Difference.columns=['click']
Difference.click.value_counts()

 0    299196
 1       327
-1       226
Name: click, dtype: int64

In [24]:
Real_accuracy=Difference.click.value_counts()[0]/(Difference.click.value_counts()[1]+Difference.click.value_counts()[-1]+
                                                 Difference.click.value_counts()[0])
Real_accuracy

0.9981551231196768

In [25]:
new_file=pd.concat([prediction_test,test],axis=1)
new_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299749 entries, 0 to 299748
Data columns (total 25 columns):
click             299749 non-null int64
weekday           299749 non-null int64
hour              299749 non-null int64
bidid             299749 non-null object
logtype           299749 non-null int64
userid            299749 non-null int64
useragent         299749 non-null int64
IP                299749 non-null object
region            299749 non-null int64
city              299749 non-null int64
adexchange        299749 non-null int64
domain            299749 non-null object
url               299749 non-null object
urlid             299749 non-null object
slotid            299749 non-null object
slotwidth         299749 non-null int64
slotheight        299749 non-null int64
slotvisibility    299749 non-null object
slotformat        299749 non-null object
slotprice         299749 non-null int64
creative          299749 non-null object
keypage           299749 non-null object

## method 1 for decision tree

In [26]:
new_file.click.value_counts()

0    299422
1       327
Name: click, dtype: int64

In [27]:
predicted_CTR=new_file.click.value_counts()[1]/(new_file.click.value_counts()[1]+new_file.click.value_counts()[0])
predicted_CTR

0.0010909127303176992

In [28]:
validation.click.value_counts()

0    299523
1       226
Name: click, dtype: int64

In [29]:
validation_CTR=validation.click.value_counts()[1]/(validation.click.value_counts()[1]+validation.click.value_counts()[0])
validation_CTR

0.00075396415000550463

## linear bidding strategy

In [30]:
bidprice_test=new_validation.bidprice*predicted_CTR/validation_CTR
bidprice_test
print ('max',":",max(bidprice_test))
print ('min',":", min(bidprice_test))

max : 434.07079646
min : 328.446902655


In [31]:
# for validation bidprice
print ('max',":",max(new_validation.bidprice))
print ('min',":", min(new_validation.bidprice))

max : 300
min : 227


In [32]:
# difference for max and min between validation and test bidprice
difference_max=max(bidprice_test)-max(new_validation.bidprice)
difference_min=min(bidprice_test)-min(new_validation.bidprice)
print ('difference for max',":",difference_max)
print ('difference for min',":",difference_min)

difference for max : 134.07079646
difference for min : 101.446902655


In [82]:
final=pd.concat([new_file, bidprice_test],axis=1)
feature=['bidid','bidprice','click']
final=final[feature]
print (sum(final[final.click==1].bidprice))
print (len(final[final.click==1].bidprice))

130519.300885
327


In [83]:
final.click.value_counts()

0    299422
1       327
Name: click, dtype: int64

In [84]:
final

,bidid,bidprice,click
0,fe2e06dff1dfd227471fd1ca717888ac,328.446903,0
1,92ea762b7b2cfd7aee30a2e88e1f5899,400.792035,0
2,617dfab714eedf4988c4aa9ad5425eca,348.703540,0
3,7fa016aaa3db39413b482f757facc3f4,348.703540,0
4,49db062ac10d19591dc7d988d730aad5,434.070796,0
5,6d86a43fbda12fc89c98620f4d8aedc1,344.362832,0
6,3b6b3f3227d42ad74971be4b9a26c189,434.070796,0
7,2cc65c3a9f97620d33d4c3ddfb368869,425.389381,0
8,be487ced968062be378892fc87013ff9,367.513274,0
9,7f7a7e763db7ec8123ddeb8a8c90689f,434.070796,0


# Adaboost

In [15]:
ada_boost=AdaBoostClassifier()
ada_boost.fit(x_validation, validation.click)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [17]:
ada_boost.score(x_validation, validation.click)

0.9992527080991096

In [18]:
prediction=ada_boost.predict(x_validation)
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

0    299747
1         2
Name: click, dtype: int64

In [26]:
test.adexchange.value_counts()

3       99997
2       93170
1       90769
4        9632
null     6181
Name: adexchange, dtype: int64

In [27]:
test.loc[test.adexchange=='3','adexchange':]=3
test.loc[test.adexchange=='2','adexchange':]=2
test.loc[test.adexchange=='1','adexchange':]=1
test.loc[test.adexchange=='4','adexchange':]=4
test.loc[test.adexchange=='null','adexchange':]=0

In [28]:
test['userid']=label_encoder.fit_transform(test['userid'])
test['useragent']=label_encoder.fit_transform(test['useragent'])
test['usertag']=label_encoder.fit_transform(test['usertag'])
test['adexchange']=label_encoder.fit_transform(test['adexchange'])

In [31]:
test['intercept']=1
feature=['useragent','usertag','userid','weekday','hour','intercept','adexchange','weekday','hour','city','region']
test_sample=test[feature]

In [35]:
prediction_test=ada_boost.predict(test_sample)
prediction_test=pd.DataFrame(prediction_test)
prediction_test.columns=['click']
prediction_test.click.value_counts()

0    299747
1         2
Name: click, dtype: int64

# KNN

In [19]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_validation, validation.click)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [20]:
accuracy=knn.score(x_validation, validation.click)
accuracy

0.9992460358499945

In [23]:
prediction=knn.predict(x_validation)
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

0    299749
Name: click, dtype: int64

# SVM

In [30]:
clf=svm.SVC()
clf.fit(x_validation, validation_1458.click)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [31]:
y_validation=validation_1458.click
accuracy=clf.score(x_validation,y_validation)
accuracy

0.99916701374427319

In [58]:
test_sample=test[test.advertiser==1458]

In [59]:
test_sample.adexchange.value_counts()

3    21855
1    20152
2    17782
Name: adexchange, dtype: int64

In [53]:
test[test.advertiser==1458].columns

Index(['weekday', 'hour', 'bidid', 'logtype', 'userid', 'useragent', 'IP',
       'region', 'city', 'adexchange', 'domain', 'url', 'urlid', 'slotid',
       'slotwidth', 'slotheight', 'slotvisibility', 'slotformat', 'slotprice',
       'creative', 'keypage', 'advertiser', 'usertag', 'intercept'],
      dtype='object')

In [60]:
test_sample['useragent']=label_encoder.fit_transform(test_sample['useragent'])


C:\Users\yifeichongtian\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [61]:
# prediction for click value in testset
test['intercept']=1
feature=['useragent','weekday','hour','intercept']
x_test=test_sample[feature]
prediction=clf.predict(x_test)
prediction

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [62]:
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

0    59789
Name: click, dtype: int64